## Basic Byte-Pair-Encoding (BPE)

In [1]:
class BasicBPE:

    def __init__(self, vocab_size: int):
        self.vocab_size = vocab_size
        self.vocab = {idx: bytes([idx]) for idx in range(256)} # BPE definition
        self.merges = dict()

    def train(self, text: str):
        """
        1. Encode text into UTF-8 format.
        2. Search for pairs and merges.
        3. Apply merges and repeat 2. until end condition.
        """

        tokens = text.encode('utf-8')
        tokens = list(map(int, tokens))

        while len(self.merges) + 256 <= self.vocab_size:
            # compute tokens stats
            tokens_stats = compute_pair_of_tokens_stats(tokens)
            # get most common pair of tokens
            most_common_pair = max(tokens_stats, key=tokens_stats.get)
            new_token_id = len(self.merges) + 1 + 256 # UTF-8 has 256 ints
            # save change
            self.merges[most_common_pair] = new_token_id
            # apply change
            tokens = merge_pair(tokens, pair=most_common_pair, idx=new_token_id)

        # update tokens map
        for (p0, p1), idx in self.merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]
    
    def encode(self, text: str):
        """
        1. Compute UTF-8 encoding of text.
        2. Apply merges to convert UTF-8 encoding to BPE encoding.
        """
        tokens = list(text.encode('utf-8'))
        while len(tokens) >= 2:
            stats = compute_pair_of_tokens_stats(tokens)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break # nothing else can be merged
            idx = self.merges[pair]
            tokens = merge_pair(tokens, pair, idx)
        return tokens

    def decode(self, tokens):
        """
        1. Convert tokens from BPE enconding to UTF-8 encoding.
        2. Decode UTF-8 to text.
        """
        text = b"".join(self.vocab[x] for x in tokens)
        text = text.decode("utf-8", errors="replace")
        return text


def compute_pair_of_tokens_stats(tokens):

    info = {}
    for pair in zip(tokens, tokens[1:]):
        if pair not in info.keys():
            info[pair] = 1
        else:
            info[pair] += 1
    return info

def merge_pair(tokens, pair, idx):
    new_list = []
    i = 0
    total_tokens = len(tokens)
    while i < total_tokens:
        if i < total_tokens - 1 and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
            new_list.append(idx)
            i += 2
        else:
            new_list.append(tokens[i])
            i += 1
    return new_list

In [47]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-14 16:31:24--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[185.199.110.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 1115394 (1,1M) [text/plain]
Guardando como: ‘input.txt’

input.txt           100%[===================>]   1,06M  --.-KB/s    en 0,09s   

2026-01-14 16:31:24 (12,3 MB/s) - ‘input.txt’ guardado [1115394/1115394]



In [25]:
text = """We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge."""#open("input.txt", "r").read()

bpe = BasicBPE(vocab_size=400)
bpe.train(text)

decoding_example = bpe.decode([270,260])
print("Decoded example: ", decoding_example)

example = "hey think we are too dear: the leanness that"
encoding_example = bpe.encode(example)
print(f"Encoded example {len(encoding_example)}: ", encoding_example)

Decoded example:  es th
Encoded example 23:  [104, 101, 121, 260, 271, 107, 32, 286, 288, 275, 111, 32, 100, 101, 266, 58, 268, 108, 101, 264, 110, 309, 292]


In [3]:
print(bpe.decode(bpe.encode("hello world")))

hello world


In [4]:
tokens_level_utf8 = text.encode("utf-8")
tokens_level_bpe = bpe.encode(text)
print(f"Number of tokens using UFT-8: {len(tokens_level_utf8)}")
print(f"Number of tokens using BPE: {len(tokens_level_bpe)}")
print(f"Compression ratio: {len(tokens_level_utf8)/len(tokens_level_bpe)*100}")

Number of tokens using UFT-8: 519
Number of tokens using BPE: 316
Compression ratio: 164.24050632911394


Considerations on increasing vocab_size

* we will increase size of the embedding table and the lm_head of the GPT model -> more computation and probable underfitting tokens since they will appear less often
* tokens will contain more text -> less "time to think" of an autoregressive model

right now, state-of-the-art is around 100k

# Regex Byte-Pair-Enconding (BPE)

In [11]:
# the main GPT text split patterns, see
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

import regex as re


class RegexBPE:

    def __init__(self, vocab_size: int):
        self.vocab_size = vocab_size
        self.vocab = {idx: bytes([idx]) for idx in range(256)} # BPE definition
        self.merges = dict()
        self.regex_pattern = GPT4_SPLIT_PATTERN
        self.regex_pattern_compiled = re.compile(self.regex_pattern)

    def train(self, text: str):
        """
        1. Encode text into UTF-8 format.
        2. Search for pairs and merges.
        3. Apply merges and repeat 2. until end condition.
        """

        spans = re.findall(self.regex_pattern_compiled, text)
        token_spans = [list(span.encode('utf-8')) for span in spans]

        while len(self.merges) + 256 <= self.vocab_size:
            # compute tokens stats
            tokens_stats = {}
            for tokens in token_spans:
                tokens_stats = compute_pair_of_tokens_stats(tokens, tokens_stats)

            # get most common pair of tokens
            most_common_pair = max(tokens_stats, key=tokens_stats.get)
            new_token_id = len(self.merges) + 1 + 256 # UTF-8 has 256 ints
            # save change
            self.merges[most_common_pair] = new_token_id
            # apply change
            token_spans = [merge_pair(tokens, pair=most_common_pair, idx=new_token_id) for tokens in token_spans]

        # update tokens map
        for (p0, p1), idx in self.merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]
    
    def encode(self, text: str):
        """
        1. Compute UTF-8 encoding of text.
        2. Apply merges to convert UTF-8 encoding to BPE encoding.
        """
        tokens = list(text.encode('utf-8'))
        while len(tokens) >= 2:
            stats = compute_pair_of_tokens_stats(tokens)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break # nothing else can be merged
            idx = self.merges[pair]
            tokens = merge_pair(tokens, pair, idx)
        return tokens

    def decode(self, tokens):
        """
        1. Convert tokens from BPE enconding to UTF-8 encoding.
        2. Decode UTF-8 to text.
        """
        text = b"".join(self.vocab[x] for x in tokens)
        text = text.decode("utf-8", errors="replace")
        return text


def compute_pair_of_tokens_stats(tokens, tokens_stats=None):

    info = {} if tokens_stats is None else tokens_stats
    for pair in zip(tokens, tokens[1:]):
        if pair not in info.keys():
            info[pair] = 1
        else:
            info[pair] += 1
    return info

def merge_pair(tokens, pair, idx):
    new_list = []
    i = 0
    total_tokens = len(tokens)
    while i < total_tokens:
        if i < total_tokens - 1 and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
            new_list.append(idx)
            i += 2
        else:
            new_list.append(tokens[i])
            i += 1
    return new_list

In [27]:
text = """We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge."""#open("input.txt", "r").read()

regex_bpe = RegexBPE(vocab_size=400)
regex_bpe.train(text)

decoding_example = regex_bpe.decode([270,260])
print("Decoded example: ", decoding_example)

example = "hey think we are too dear: the leanness that"
encoding_example = regex_bpe.encode(example)
print(f"Encoded example {len(encoding_example)}: ", encoding_example)

Decoded example:   usre
Encoded example 18:  [104, 101, 121, 398, 277, 293, 399, 401, 288, 58, 259, 32, 302, 267, 110, 306, 258, 278]


In [29]:
bpe.vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [30]:
regex_bpe.vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'